In [4]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [5]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Data
import xarray as xr
import h5py
import pickle

# Helpful
import time
import datetime
import itertools
from itertools import product
from tqdm import tqdm
import os

# My Methods
import importlib
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
from src.models.EMOS_global.EMOS_global_load_models import *

<IPython.core.display.Javascript object>

## Goal: Prepare everything for EMOS_local

### 0. Basics

In [6]:
var_names = ["u10", "v10", "t2m", "t850", "z500"]

<IPython.core.display.Javascript object>

In [7]:
# Variables to make testing easier --> determines variable and lead_time
var = 0  # 0,...,4
lead_time = 0  # 0,...,30

<IPython.core.display.Javascript object>

### 1. Load Dataset

In [8]:
dat_train_proc_norm = ldp.load_data_all_train_proc_norm()
dat_test_proc_norm = ldp.load_data_all_test_proc_norm()

<IPython.core.display.Javascript object>

### 2. Datasplit

In [12]:
# In the moment:
X_train_var_lead_all, y_train_var_lead_all = split_var_lead(dat_train_proc_norm)
X_test_lead_all, y_test_var_lead_all = split_var_lead(dat_test_proc_norm)

<IPython.core.display.Javascript object>

Goal: Split training and test datatset inot different datasets with var_lead_lat_lon

In [26]:
dat_train_proc_norm[0][list(dat_train_proc_norm[0].data_vars.keys())[0]]

<xarray.DataArray 'u10_train' (forecast_date: 1429, lead_time: 32, lat: 120,
                               lon: 130, mean_std: 2)>
[1426713600 values with dtype=float32]
Dimensions without coordinates: forecast_date, lead_time, lat, lon, mean_std

<IPython.core.display.Javascript object>

In [50]:
def split_var_lead_lat_lon(dat):
    dat_X_var_lead_lat_lon = []
    dat_y_var_lead_lat_lon = []
    for var in tqdm(range(5)):
        X_var = []
        y_var = []
        for lead_time in range(1, 31):
            X_lead_time = []
            y_lead_time = []
            for lat in range(120):
                X_lat = []
                y_lat = []
                for lon in range(130):
                    #                     print(dat[var][list(dat[var].data_vars.keys())[0]])
                    X_lat.append(
                        dat[var][list(dat[var].data_vars.keys())[0]].isel(
                            lead_time=lead_time, lat=lat, lon=lon
                        )
                    )
                    y_lat.append(
                        dat[var][list(dat[var].data_vars.keys())[1]].isel(
                            lead_time=lead_time, lat=lat, lon=lon
                        )
                    )
                X_lead_time.append(X_lat)
                y_lead_time.append(y_lat)
            X_var.append(X_lead_time)
            y_var.append(y_lead_time)
        dat_X_var_lead_lat_lon.append(X_var)
        dat_y_var_lead_lat_lon.append(y_var)
    return dat_X_var_lead_lat_lon, dat_y_var_lead_lat_lon

<IPython.core.display.Javascript object>

In [51]:
start = time.time()
X_train_var_lead_lat_lon, y_train_var_lead_lat_lon = split_var_lead_lat_lon(
    dat_train_proc_norm
)
print(f"Time after first process: {time.time() - start} seconds")
X_test_var_lead_lat_lon, y_test_var_lead_lat_lon = split_var_lead_lat_lon(
    dat_train_proc_norm
)
print(f'Time after second process: {time.time() - start} seconds')

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [04:37<00:00, 55.55s/it]

Time after first process: 277.74137020111084 seconds


<IPython.core.display.Javascript object>